# 나이브 베이즈(Naive Bayes)

## 01 나이브 베이즈 개요

### 나이브 베이즈 분류기 특징

- 사전 확률 및 추가 정보를 기반으로 사후 확률을 추론하는 통계적 방법인 베이즈 추정 기반 분류
- 종속변수 각 범우즤 등장 빈도인 사전확률(prior) 설정이 중요
- 각 데이터의 사전 확률을 기반으로 사후확률(posterior)을 계산

## 02 주요 함수 및 메서드 소개

### sklearn - GaussianNB()

- 나이브베이즈 분류 모델을 위한 sklearn의 함수
- 독립변수와 종속변수는 GaussianNB() 함수의 메서드인 fit() 함수에 할당
- 모델 객체의 predict_proba() 메서드로 예측 확률값을 생산
- 이진 분류의 경우 출력된 예측 확률값의 두 번째 열이 1이 될 확률

In [2]:
import pandas as pd
from sklearn.naive_bayes import GaussianNB

In [2]:
df = pd.read_csv("실습파일/iris.csv")
df.head()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [5]:
df["is_setosa"] = (df["Species"] == "setosa") + 0
df.head()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,is_setosa
0,5.1,3.5,1.4,0.2,setosa,1
1,4.9,3.0,1.4,0.2,setosa,1
2,4.7,3.2,1.3,0.2,setosa,1
3,4.6,3.1,1.5,0.2,setosa,1
4,5.0,3.6,1.4,0.2,setosa,1


In [6]:
df["is_setosa"].value_counts()

0    100
1     50
Name: is_setosa, dtype: int64

In [7]:
df["is_setosa"].value_counts(normalize = True)  # 사전 확률

0    0.666667
1    0.333333
Name: is_setosa, dtype: float64

In [9]:
model = GaussianNB().fit(X = df.iloc[:, :4],
                        y = df["is_setosa"])
model

GaussianNB()

In [10]:
model.class_prior_

array([0.66666667, 0.33333333])

In [11]:
model.theta_

array([[6.262, 2.872, 4.906, 1.676],
       [5.006, 3.428, 1.462, 0.246]])

In [14]:
pred = model.predict_proba(df.iloc[:, :4])
pred = pred[:, 1]
pred[:4]

array([1., 1., 1., 1.])

In [7]:
from sklearn.metrics import accuracy_score

In [16]:
pred_class = (pred > 0.5) + 0

In [17]:
accuracy_score(y_true = df["is_setosa"],
              y_pred = pred_class)

1.0

## Q1 BMI가 0 초과인 데이터만 사용하여 나이브 베이즈 분류를 실시하고자 한다. Outcome을 종속변수로 하고 나머지 변수를 독립변수로 할 때 종속변수의 사전확률은?
1) Outcome 1을 대상으로 사전확률을 계산한다.

In [27]:
Q1 = pd.read_csv("실습파일/diabetes.csv")
Q1 = Q1[Q1["BMI"] > 0]
Q1.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [28]:
Q1["Outcome"].value_counts(normalize = True)

0    0.648613
1    0.351387
Name: Outcome, dtype: float64

In [29]:
Q1_model = GaussianNB().fit(X = Q1.iloc[:, :-1],
                        y = Q1["Outcome"])
Q1_model.class_prior_

array([0.64861295, 0.35138705])

## Q2 혈당, 혈압, 나이를 독립변수로 하고 당뇨 발병 여부를 종속변수로 했을 때 그 정확도는 얼마인가?

In [38]:
Q2_model = GaussianNB().fit(X = Q1.loc[:, ["Glucose", "BloodPressure", "Age"]],
                        y = Q1["Outcome"])
Q2_pred = Q2_model.predict_proba(Q1.loc[:, ["Glucose", "BloodPressure", "Age"]])

In [40]:
Q2_pred_class = (Q2_pred[:, 1] > 0.5) +0
Q2_pred_class[:4]

array([1, 0, 1, 0])

In [42]:
accuracy_score(y_pred = Q2_pred_class, y_true = Q1["Outcome"])

0.7490092470277411

## Q3 임신여부, 연령대, BMI, 혈당을 독립변수로 하고 당뇨 발병 여부를 종속변수로 했을 때 나이브 베이즈와 로지스틱 회귀 분석을 실시하고 둘 중 정확도가 높은 모델의 정확도는?
1) BMI가 0 초과인 것을 사용하며 학습/평가 데이터 세트를 8:2로 분할, Seed는 123  
2) 연령대는 Age가 21인 경우 20으로, 39일 경우 30으로 계산한다.  
3) sklearn의 로지스틱 회귀 함수를 사용하며, 임계값(threshold)은 0.5로 한다.

In [3]:
Q3 = pd.read_csv("실습파일/diabetes.csv")
Q3.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
Q3 = Q3.loc[Q3["BMI"] > 0. ]
Q3["Age_g"] = (Q3["Age"] // 10) * 10
Q3["is_preg"] = (Q3["Pregnancies"] > 0) + 0
Q3.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Age_g,is_preg
0,6,148,72,35,0,33.6,0.627,50,1,50,1
1,1,85,66,29,0,26.6,0.351,31,0,30,1
2,8,183,64,0,0,23.3,0.672,32,1,30,1
3,1,89,66,23,94,28.1,0.167,21,0,20,1
4,0,137,40,35,168,43.1,2.288,33,1,30,0


In [5]:
from sklearn.model_selection import train_test_split
Q3_train, Q3_test = train_test_split(Q3, train_size = 0.8, random_state = 123)
Q3_train.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Age_g,is_preg
247,0,165,90,33,680,52.3,0.427,23,0,20,0
659,3,80,82,31,70,34.2,1.292,27,1,20,1
156,2,99,52,15,94,24.6,0.637,21,0,20,1
254,12,92,62,7,258,27.6,0.926,44,1,40,1
62,5,44,62,0,0,25.0,0.587,36,0,30,1


In [6]:
Q3_model = GaussianNB().fit(X = Q3_train.loc[:, ["is_preg", "Age_g", "BMI", "Glucose"]],
                           y = Q3_train["Outcome"])
Q3_pred = Q3_model.predict_proba(Q3_test.loc[:, ["is_preg", "Age_g", "BMI", "Glucose"]])
Q3_pred[:4,]

array([[0.09436402, 0.90563598],
       [0.74783283, 0.25216717],
       [0.11042961, 0.88957039],
       [0.57991266, 0.42008734]])

In [10]:
accuracy_score(y_pred = (Q3_pred[:, 1] > 0.5) +0,
              y_true = Q3_test["Outcome"])

0.8026315789473685

In [11]:
from sklearn.linear_model import LogisticRegression

In [12]:
Q3_model_lr = LogisticRegression()
Q3_model_lr.fit(X = Q3_train.loc[:, ["is_preg", "Age_g", "BMI", "Glucose"]],
                           y = Q3_train["Outcome"])

LogisticRegression()

In [13]:
Q3_pred_lr = Q3_model_lr.predict_proba(Q3_test.loc[:, ["is_preg", "Age_g", "BMI", "Glucose"]])
Q3_pred_lr = Q3_pred_lr[:, 1]
Q3_pred_lr_class = (Q3_pred_lr > 0.5) + 0

In [14]:
accuracy_score(y_true = Q3_test["Outcome"],
              y_pred = Q3_pred_lr_class)

0.8289473684210527